In [2]:
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
from geopy.geocoders import Nominatim

## Geotagging the scraped Data

In [3]:
df = pd.read_csv('all_reviews.csv')
locations_df = pd.read_csv('locations_geotagged.csv')
beer_df = pd.read_csv('beer_deets.csv')

In [4]:
df

,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit
...,...,...,...,...,...,...
27911,Oakham Green Devil IPA,1.5,-2.2,Oakham Ales,Was on offer in the local so tried it. This is...,SampleScout19
27912,Deschutes Botanic Ale,1.6,-1.6,Deschutes Brewery,"Bottle on clearance at bowsers, cheap. Cloudy ...",cheap
27913,Queen City Yorkshire Porter,1.3,-2.1,Queen City Brewery,"Thin bodied, almost no foam or head, zero laci...",TigerSoul
27914,Wild Beer Nebula,1.8,-1.7,Wild Beer,"Keg at We Brought Beer, Balham. Hazy straw wi...",Beese


In [5]:
locations_df

,locations,latitude,longitude,country
0,"Bungay, Suffolk",52.456225,1.437341,United Kingdom
1,"Faversham, Kent",51.314409,0.891189,United Kingdom
2,"Bristol, Gloucestershire",51.852767,-2.255369,United Kingdom
3,"Round Rock, Texas",30.508235,-97.678893,United States of America
4,"Brooklyn, New York",40.650104,-73.949582,United States of America
...,...,...,...,...
2209,"Bielefeld, North Rhine-Westphalia",52.019101,8.531007,Germany
2210,"Truro, Cornwall",50.262752,-5.052047,United Kingdom
2211,"Smethwick, West Midlands",52.496816,-1.973917,United Kingdom
2212,"Newton Poppleford, Devon",50.700961,-3.295237,United Kingdom


In [6]:
geolocator = Nominatim(user_agent="None")

In [7]:
locations_new = pd.DataFrame(df.brewery_location[~df.brewery_location.isin(locations_df.locations)].unique(),columns=['locations'])
print(len(locations_new), "new locations to search for! Will take", len(locations_new)/2, "seconds to run.")

137 new locations to search for! Will take 68.5 seconds to run.


In [8]:
def tryconvert(x):
        try:
            return geolocator.geocode(x).latitude
        except:
            return None
    

In [9]:
def tryconvertlong(x):
        try:
            return geolocator.geocode(x).longitude
        except:
            return None

In [10]:
locations_new["latitude"] = locations_new.locations.apply(lambda x: tryconvert(x))

In [11]:
locations_new["longitude"] = locations_new.locations.apply(lambda x: tryconvertlong(x))

In [12]:
countries = []
for row in locations_new.index:
    try:
        loc = geolocator.reverse([locations_new.iloc[row,1],locations_new.iloc[row,2]], language='en')
        countries.append(loc.raw['address']['country'])
    except:
        countries.append(None)
        
    

In [13]:
locations_new['country'] = countries

In [14]:
locations_new

,locations,latitude,longitude,country
0,"Winooski, Vermont",44.491040,-73.186106,United States of America
1,"Hyde Park, Vermont",44.593941,-72.616505,United States of America
2,"Flagstaff, Arizona",35.198752,-111.651823,United States of America
3,"Mars, Pennsylvania",40.695834,-80.011878,United States of America
4,"Granbury, Texas",32.440779,-97.792609,United States of America
...,...,...,...,...
132,"Rockville, Virginia",39.621471,-79.704774,United States of America
133,"Axberg, Austria",46.953826,15.760800,Austria
134,"Royal Oak, Michigan",42.489480,-83.144649,United States of America
135,"Bécancour, Quebec",46.339343,-72.433205,Canada


In [15]:
locations_df = pd.concat([locations_df,locations_new])
locations_df.to_csv('locations_geotagged.csv',index=False)

## Merging and cleaning

In [16]:
beer_df

,return_string,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number,name_found
0,St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...,🇬🇧,Bitter - Ordinary / Best,4.3,49,95,3.25,415.0,St. Peter's Ruby Red Ale
1,Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...,🇬🇧,Bitter - Ordinary / Best,4.5,34,59,3.05,1083.0,Shepherd Neame Spitfire (Bottle)
2,Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...,🇬🇧,IPA,6.0,95,97,3.74,59.0,Arbor Rocketman
3,Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...,🇬🇧,IPA - English,6.1,46,47,3.22,420.0,Shepherd Neame India Pale Ale (Bottle)
4,Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...,🇬🇧,Strong Ale - English,7.0,45,46,3.18,409.0,Shepherd Neame Christmas Ale
...,...,...,...,...,...,...,...,...,...
14899,4 Noses Something Hip\n🇺🇸 Sour / Wild Beer\n-\...,🇺🇸,Sour / Wild Beer,NaN,-,-,3.00,1.0,4 Noses Something Hip
14900,Casey NZ State of Mind\n🇺🇸 IPA - Hazy / NEIPA\...,🇺🇸,IPA - Hazy / NEIPA,NaN,-,-,3.80,1.0,Casey NZ State of Mind
14901,Eve's New Field Northern Spy 2017\n🇺🇸 Apple Ci...,🇺🇸,Apple Cider,NaN,-,-,4.40,1.0,Eve's New Field Northern Spy 2017
14902,Cycle Cherry Coffee BA Imperial Stout\n🇺🇸 Stou...,🇺🇸,Stout - Imperial,0.0,-,-,4.20,7.0,Cycle Cherry Coffee BA Imperial Stout


In [17]:
# Creating DF with all details
df_all = pd.merge(df,locations_df,how='left',left_on='brewery_location',right_on='locations')
df_all = pd.merge(df_all, beer_df, how='left', left_on='beer_name',right_on='name_found')
df_all


,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name,locations,latitude,longitude,country,return_string,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number,name_found
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit,"Bungay, Suffolk",52.456225,1.437341,United Kingdom,St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...,🇬🇧,Bitter - Ordinary / Best,4.3,49,95,3.25,415.00,St. Peter's Ruby Red Ale
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray,"Faversham, Kent",51.314409,0.891189,United Kingdom,Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...,🇬🇧,Bitter - Ordinary / Best,4.5,34,59,3.05,1083.00,Shepherd Neame Spitfire (Bottle)
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG,"Bristol, Gloucestershire",51.852767,-2.255369,United Kingdom,Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...,🇬🇧,IPA,6.0,95,97,3.74,59.00,Arbor Rocketman
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit,"Faversham, Kent",51.314409,0.891189,United Kingdom,Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...,🇬🇧,IPA - English,6.1,46,47,3.22,420.00,Shepherd Neame India Pale Ale (Bottle)
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit,"Faversham, Kent",51.314409,0.891189,United Kingdom,Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...,🇬🇧,Strong Ale - English,7.0,45,46,3.18,409.00,Shepherd Neame Christmas Ale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28183,Oakham Green Devil IPA,1.5,-2.2,Oakham Ales,Was on offer in the local so tried it. This is...,SampleScout19,-2.2,48.793281,9.967117,Germany,Oakham Green Devil IPA\n🇬🇧 IPA • 6.0%\n96\n97\...,🇬🇧,IPA,6.0,96,97,3.78,435.00,Oakham Green Devil IPA
28184,Deschutes Botanic Ale,1.6,-1.6,Deschutes Brewery,"Bottle on clearance at bowsers, cheap. Cloudy ...",cheap,-1.6,48.718182,9.989251,Germany,Deschutes Botanic Ale\n🇺🇸 Flavored - Other • 5...,🇺🇸,Flavored - Other,5.5,49,79,3.37,18.00,Deschutes Botanic Ale
28185,Queen City Yorkshire Porter,1.3,-2.1,Queen City Brewery,"Thin bodied, almost no foam or head, zero laci...",TigerSoul,-2.1,33.814488,130.890462,Japan,Queen City Yorkshire Porter\n🇺🇸 Porter • 5.0%\...,🇺🇸,Porter,5.0,77,77,3.45,29.00,Queen City Yorkshire Porter
28186,Wild Beer Nebula,1.8,-1.7,Wild Beer,"Keg at We Brought Beer, Balham. Hazy straw wi...",Beese,-1.7,48.780313,10.039726,Germany,Wild Beer Nebula\n🇬🇧 IPA - Hazy / NEIPA • 5.0%...,🇬🇧,IPA - Hazy / NEIPA,5.0,Available from £2.33,88,50,3.61,Wild Beer Nebula


In [18]:
# drop excess columns
df_all.drop(["locations","return_string","name_found"],axis=1,inplace=True)
# drop rows with ratings instead of location
df_all = df_all[~df_all['brewery_location'].str.len()<4]
# drop all NaNs
print(len(df_all) - len(df_all.dropna()), "columns with NaNs dropped.")
df_all.dropna(inplace=True)


2138 columns with NaNs dropped.


In [19]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26050 entries, 0 to 28187
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   beer_name         26050 non-null  object 
 1   beer_rating       26050 non-null  float64
 2   brewery_location  26050 non-null  object 
 3   brewery_name      26050 non-null  object 
 4   review_content    26050 non-null  object 
 5   reviewer_name     26050 non-null  object 
 6   latitude          26050 non-null  float64
 7   longitude         26050 non-null  float64
 8   country           26050 non-null  object 
 9   flag              26050 non-null  object 
 10  type              26050 non-null  object 
 11  abv               26050 non-null  float64
 12  rb_overall_score  26050 non-null  object 
 13  rb_style_score    26050 non-null  object 
 14  rb_user_rating    26050 non-null  object 
 15  rb_rating_number  26050 non-null  float64
dtypes: float64(5), object(11)
memory usage: 

## Deal with Non-English Descriptions

## Save down cleaned data

In [20]:
df_all.to_csv("data_clean.csv",index=False)